In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
from selenium import webdriver
from datetime import datetime,date

In [2]:
#step 2 ===> Data Gathering / Data Extracting
# -> Let’s now create a new instance of google chrome.
# -> This will help our program open an url in google chrome & dynamically Scrap our data (The reason we are using selenium)

driver = webdriver.Chrome(executable_path = r"/home/programmer-0/Data-Science/DataAnalysisCovidIndia/chromedriver")
url = 'https://www.mohfw.gov.in/'
driver.get(url) #using selenium for webpage table content because it is dynamically loads after page loads because of javascript(DOM)
webContent = driver.page_source
PySoup = BeautifulSoup(webContent,'html')
driver.quit()

In [3]:
PySoup

<html><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="0" http-equiv="Expires"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>MoHFW | Home</title>
<link href="assets/images/favicon.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="assets/css/base.css" rel="stylesheet" type="text/css"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
</head><body><iframe src="javascript:void(0)" style="width: 0px; height: 0px; border: 0px; display: none;" title=""></iframe><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-162536712-2"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new D

In [4]:
div_data = PySoup.find_all('div',{'class' : "data-table table-responsive"})
get_date = div_data[0].h5.span.text.split(":")[1].split(",")[0]
format = ' %d %B %Y' # The format we want to respresent our date 
datetime_obj = datetime.strptime(get_date, format).date()
print(type(datetime_obj))

<class 'datetime.date'>


In [5]:
#Scraping headings ==>
get_head = PySoup.find_all('thead')[0].find('tr',{'class' : "row1"})
headings = [th.get_text().strip() for th in get_head.find_all("th")]
headings.insert(3,'Change Active Case Since Yesterday')
headings.insert(5,'Change Cured Case Since Yesterday')
headings.insert(7,'Change Death Case Since Yesterday')

In [6]:
get_tbody = PySoup.find_all('tbody')[0]      #find_all returns list of tags that contains data
#Extracting Table Data into "mytabledata" [] ==>
mytabledata = []
list_of_tr_rows = get_tbody.find_all("tr")
for tr_row in list_of_tr_rows:
    if isinstance(tr_row,NavigableString):
        continue
    if isinstance(tr_row, Tag):
        statedata_singlerow = [-int(td.get_text().strip()) \
                               if (td.span != None) and (td.span.get('class',[0])[0] == "down") \
                               else td.get_text().strip() \
                               for td in tr_row.find_all("td")
                              ]
        data = dict(zip( headings,(statedata_singlerow) ))  
        if 'Active Cases*' in data:
            mytabledata.append(data)

In [7]:
state_data = pd.DataFrame(mytabledata, columns=headings)
new_cols = ["Sr.No","States/UT","Active Cases","Active Cases Since Yesterday",\
            "Recovered","Recovered Cases Since Yesterday","Deceased","Deceased Cases Since Yesterday"]
state_data.columns = new_cols
state_data = state_data.set_index('Sr.No')
state_data.loc['Total#']['Deceased'] = state_data.loc['Total#']['Recovered']
state_data.loc['Total#']['Recovered'] = state_data.loc['Total#']['Active Cases']
state_data.loc['Total#']['Active Cases'] = state_data.loc['Total#']['States/UT']
print(str(datetime_obj.strftime("%d-%m-%Y")))
print(type(str(datetime_obj.strftime("%d-%m-%Y"))))
state_data['Date'] = str(datetime_obj) #.strftime("%d-%m-%Y"))
print(type(state_data['Date']))
# state_data['Date'] = datetime.now().date()

# state_data['Date'] = pd.to_datetime(state_data['Date'])
# df_new = state_data.rename(columns={'A': 'a'}, index={'ONE': 'one'})   #for renaming columns
state_data.rename(index={'Total#': 'Total'},inplace=True)

06-01-2021
<class 'str'>
<class 'pandas.core.series.Series'>


In [8]:
state_data.replace(['',np.nan], 0,inplace=True)
cols_to_include =[i for i in state_data.columns if i not in ['States/UT','Date']]
state_data[cols_to_include] = state_data[cols_to_include].astype('int')
state_data['States/UT'] = state_data['States/UT'].astype('str')
state_data['Deceased Cases Since Yesterday'] = state_data['Deceased Cases Since Yesterday'].abs()
# state_data.loc['35']['Deceased Cases Since Yesterday'] = sum(state_data['Deceased Cases Since Yesterday'])
state_data.iat[35, 6] = sum(state_data['Deceased Cases Since Yesterday'])
# state_data.at[34,'Deceased Cases Since Yesterday'] = sum(state_data['Deceased Cases Since Yesterday'])
# print(state_data['Deceased Cases Since Yesterday'][0])# = sum(state_data['Deceased Cases Since Yesterday'])

state_data

,States/UT,Active Cases,Active Cases Since Yesterday,Recovered,Recovered Cases Since Yesterday,Deceased,Deceased Cases Since Yesterday,Date
Sr.No,,,,,,,,
1,Andaman and Nicobar Islands,31,-7,4856,8,62,0,2021-01-06
2,Andhra Pradesh,3038,95,873427,278,7122,4,2021-01-06
3,Arunachal Pradesh,79,-8,16609,11,56,0,2021-01-06
4,Assam,3075,-12,212249,50,1057,3,2021-01-06
5,Bihar,4351,-29,247929,445,1415,7,2021-01-06
6,Chandigarh,240,-27,19464,116,322,1,2021-01-06
7,Chhattisgarh,9111,-655,271988,1651,3437,25,2021-01-06
8,Dadra and Nagar Haveli and Daman and Diu,7,-1,3370,1,2,0,2021-01-06
9,Delhi,4562,-127,612527,557,10609,12,2021-01-06
